<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Xarray and Dask

## Table of contents

2. Parallel/streaming/lazy computation using dask.array with Xarray
3. Reading and writing data with Dask and Xarray
4. Automatic parallelization with apply_ufunc and map_blocks


In [1]:
from dask.distributed import Client

client = Client()
client

Client Scheduler: tcp://127.0.0.1:37727 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 8.36 GB


<p>&#128070</p> Click the Dashboard link above.

## Parallel/streaming/lazy computation using dask.array with Xarray


## Reading and writing data with Dask and Xarray


## Automatic parallelization with apply_ufunc and map_blocks
